In [ ]:
def check(nerd):
    left, right = nerd.split("=")
    try:
        return compute(left)==compute(right)
    except ZeroDivisionError:
        return False

def compute(side):
    if "+" in side or "-" in side:
        ix = max(side.rfind("+"), side.rfind("-"))
        if side[ix]=="+":
            return compute(side[:ix]) + compute(side[ix+1:])
        elif side[ix]=="-":
            return compute(side[:ix]) - compute(side[ix+1:])
        assert False
    elif "*" in side or "/" in side:
        ix = max(side.rfind("*"), side.rfind("/"))
        if side[ix]=="*":
            return compute(side[:ix]) * compute(side[ix+1:])
        elif side[ix]=="/":
            return compute(side[:ix]) / compute(side[ix+1:])
        assert False
    return float(side)

In [ ]:
import random

def random_side(k, forbidden="", offset=0, exact={}):
    possible_opchars = set("-+*/") - set(forbidden)
    new_lenght_possiblities = set(range(1,k-1)) - {_k-offset for _k in exact.keys()}
    forcesplit = False
    for _k in filter(lambda x:x>=offset and x<k-offset, exact.keys()):
        if exact[_k] in set("-+*/"):
            forcesplit = True
            new_lenght = _k - offset
            possible_opchars = {exact[_k]}
    if forcesplit or (len(new_lenght_possiblities)>0 and len(possible_opchars)>0 and random.randint(0,1)==0):
        new_lenght = random.choice(list(new_lenght_possiblities)) if not forcesplit else new_lenght
        return random_side(new_lenght, forbidden=forbidden, offset=offset, exact=exact) + random.choice(list(possible_opchars)) + random_side(k-new_lenght-1, forbidden=forbidden, offset=offset+new_lenght+1, exact=exact)
    chars = random.choices(list(set("0123456789") - set(forbidden)), k=k)
    for ix in range(len(chars)):
        if ix+offset in exact:
            chars[ix] = exact[ix+offset]
    return "".join(chars)

def random_nerdle(k:int=8, forbidden:str="", exact:dict={}):
    ixpos = set(range(1,k-1))
    for _k in filter(lambda x:exact.get(x) in set("*/+-"), exact):
        ixpos -= {_k-1, _k, _k+1}
    ixeq = random.choice(list(ixpos))
    for _k,v in exact.items():
        if v=="=":
            ixeq=_k
    return random_side(k=ixeq, forbidden=forbidden, offset=0, exact=exact) + "=" + random_side(k=k-ixeq-1, forbidden=forbidden, offset=ixeq+1, exact=exact)

In [ ]:
def history_check(nerd_in, history):
    for nerd_hist, nerd_tuple  in history.items():
        nerd_in2=""
        chars = {0: set(), 1:set()}
        for char_in, char_hist, digit in zip(nerd_in, nerd_hist, nerd_tuple):
            if char_in==char_hist and digit!=2:
                return False
            elif char_in!=char_hist and digit==2:
                return False
            elif digit!=2:
                nerd_in2+=char_in
                chars[digit].add(char_hist)
        if len(set(nerd_in2) & chars[0])>0:
            return False
        if len(chars[1]-set(nerd_in2))>0:
            return False
    return True

In [ ]:
def valid_nerdle(k=8, history={}):
    chars = {i:set() for i in range(3)}
    [chars[digit].add(char)for nerd_hist,nerd_tuple in history.items() for (char, digit) in zip(nerd_hist, nerd_tuple)]
    exact = {ix:char for (nerd_hist,nerd_tuple) in history.items() for ix,(char, digit) in enumerate(zip(nerd_hist, nerd_tuple)) if digit==2}
    nerdle = random_nerdle(k=k, forbidden=chars[0]-chars[2], exact=exact)
    while not check(nerdle) or not history_check(nerdle, history):
        nerdle = random_nerdle(k=k, forbidden=chars[0]-chars[2], exact=exact)
    return nerdle

In [ ]:
# 0: out
# 1: in
# 2: exact

history = {
    '21-9=4*3': (1,1,0,0,1,0,1,0),
}

sorted({valid_nerdle(k=8, history=history) for _ in range(10)}, key=lambda x:len(set(x)), reverse=True)